Importing Necessary Libraries

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import os
from torchsummary import summary
import tensorflow as tf

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout

In [3]:
import torchvision
torchvision.__version__

'0.13.0+cpu'

Specify the path where the data is present

In [4]:
#import splitfolders
#splitfolders.ratio("Curated X-Ray Dataset", output="X_ray_data", seed=1337, ratio=(.8, .1, .1), group_prefix=None, move=False)

In [5]:
input_path = "X_ray_data/"

Create Pytorch Generators

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder(input_path + 'val', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  
}

In [7]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 7364
     Root location: X_ray_data/train
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
                RandomAffine(degrees=[0.0, 0.0], scale=(0.8, 1.2), shear=[-10.0, 10.0])
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'validation': Dataset ImageFolder
     Number of datapoints: 920
     Root location: X_ray_data/val
     StandardTransform
 Transform: Compose(
                Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [14]:
model = models.resnet50(weights = 'ResNet50_Weights.DEFAULT').to(device)
i = 50
for param in model.parameters():
    i-=1
    if(i <= 1):
        break
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 4)).to(device)

In [15]:
summary(model, (3,244,244))
##For initial model before unfreezing few layers in resnet50
##Total params: 23,770,820
##Trainable params: 262,788
##Non-trainable params: 23,508,032

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]           4,096
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]          16,384
      BatchNorm2d-12          [-1, 256, 61, 61]             512
           Conv2d-13          [-1, 256, 61, 61]          16,384
      BatchNorm2d-14          [-1, 256,

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [19]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])
            if(epoch_acc >= 0.98):
                raise
            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [20]:
%%time
model_trained = train_model(model, criterion, optimizer, num_epochs=4)

Epoch 1/4
----------
train loss: 0.4012, acc: 0.8281
validation loss: 0.4894, acc: 0.8022
Epoch 2/4
----------
train loss: 0.3881, acc: 0.8388
validation loss: 0.3746, acc: 0.8337
Epoch 3/4
----------
train loss: 0.3506, acc: 0.8514
validation loss: 0.3669, acc: 0.8348
Epoch 4/4
----------
train loss: 0.3460, acc: 0.8561
validation loss: 0.3434, acc: 0.8533
Wall time: 2h 10min 1s


In [21]:
%%time
model_trained = train_model(model, criterion, optimizer, num_epochs=15)

Epoch 1/15
----------
train loss: 0.3488, acc: 0.8512
validation loss: 0.3714, acc: 0.8522
Epoch 2/15
----------
train loss: 0.3149, acc: 0.8638
validation loss: 0.3833, acc: 0.8304
Epoch 3/15
----------
train loss: 0.3043, acc: 0.8683
validation loss: 0.3320, acc: 0.8565
Epoch 4/15
----------
train loss: 0.3030, acc: 0.8695
validation loss: 0.3495, acc: 0.8522
Epoch 5/15
----------
train loss: 0.2890, acc: 0.8760
validation loss: 0.3524, acc: 0.8565
Epoch 6/15
----------
train loss: 0.2880, acc: 0.8766
validation loss: 0.3141, acc: 0.8717
Epoch 7/15
----------
train loss: 0.3015, acc: 0.8709
validation loss: 0.3928, acc: 0.8217
Epoch 8/15
----------
train loss: 0.2854, acc: 0.8772
validation loss: 0.3185, acc: 0.8717
Epoch 9/15
----------
train loss: 0.2797, acc: 0.8791
validation loss: 0.2856, acc: 0.8783
Epoch 10/15
----------
train loss: 0.2686, acc: 0.8858
validation loss: 0.3923, acc: 0.8478
Epoch 11/15
----------
train loss: 0.2651, acc: 0.8851
validation loss: 0.2772, acc: 0.88

In [15]:
model = models.resnet50(weights = 'ResNet50_Weights.IMAGENET1K_V2').to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [45]:
model = models.resnet50(weights = 'ResNet50_Weights.IMAGENET1K_V2').to(device)
i = 25
for param in model.parameters():
    i-=1
    if(i <= 1):
        break
    param.requires_grad = False  
model.fc = nn.Sequential(
               nn.Linear(2048, 1024),
               nn.Linear(1024, 512),
               nn.Linear(512, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 4)).to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [46]:
summary(model, (3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]           4,096
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]          16,384
      BatchNorm2d-12          [-1, 256, 61, 61]             512
           Conv2d-13          [-1, 256, 61, 61]          16,384
      BatchNorm2d-14          [-1, 256,

In [47]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
%%time
model_trained = train_model(model, criterion, optimizer, num_epochs=15)

Epoch 1/15
----------
train loss: 0.5977, acc: 0.7546
validation loss: 0.5876, acc: 0.8043
Epoch 2/15
----------
train loss: 0.4519, acc: 0.8144
validation loss: 0.4465, acc: 0.8293
Epoch 3/15
----------
train loss: 0.4375, acc: 0.8212
validation loss: 0.3562, acc: 0.8467
Epoch 4/15
----------
train loss: 0.4070, acc: 0.8266
validation loss: 0.3474, acc: 0.8587
Epoch 5/15
----------
train loss: 0.3914, acc: 0.8334
validation loss: 0.3805, acc: 0.8250
Epoch 6/15
----------
train loss: 0.3824, acc: 0.8385
validation loss: 0.4180, acc: 0.8250
Epoch 7/15
----------
train loss: 0.3619, acc: 0.8457
validation loss: 0.3548, acc: 0.8478
Epoch 8/15
----------
train loss: 0.3613, acc: 0.8437
validation loss: 0.3197, acc: 0.8630
Epoch 9/15
----------
train loss: 0.3486, acc: 0.8493
validation loss: 0.3724, acc: 0.8228
Epoch 10/15
----------
train loss: 0.3451, acc: 0.8537
validation loss: 0.4074, acc: 0.8457
Epoch 11/15
----------
train loss: 0.3492, acc: 0.8523
validation loss: 0.3435, acc: 0.85

In [69]:
model = models.resnet50(weights = 'ResNet50_Weights.IMAGENET1K_V2').to(device)
i = 150
for param in model.parameters():
    i-=1
    if(i <= 1):
        param.requires_grad = False   
model.fc = nn.Sequential(
               nn.Linear(2048, 512),
               nn.ReLU(inplace=True),
               nn.Linear(512, 4)).to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [70]:
summary(model, (3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]           4,096
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]          16,384
      BatchNorm2d-12          [-1, 256, 61, 61]             512
           Conv2d-13          [-1, 256, 61, 61]          16,384
      BatchNorm2d-14          [-1, 256,

In [54]:
%%time
model_trained = train_model(model, criterion, optimizer, num_epochs=3)

Epoch 1/3
----------
train loss: 1.3842, acc: 0.2917
validation loss: 1.3851, acc: 0.2902
Epoch 2/3
----------
train loss: 1.3838, acc: 0.2970
validation loss: 1.3851, acc: 0.2783
Epoch 3/3
----------
train loss: 1.3841, acc: 0.2982
validation loss: 1.3856, acc: 0.2880
Wall time: 15h 1min 51s


In [42]:
c = 0
for param in model.parameters():
    c+=1
print(c)

163
